In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import uuid
import scipy.spatial.distance as dist

from abc import ABC, abstractmethod

In [2]:
class StreetSegment:
    pass

In [3]:
class Node:

    node_list = []

    def __init__(self, coordinates: tuple, logo: str) -> None:
        self.coordinates = coordinates
        self.x = coordinates[0]
        self.y = coordinates[1]
        self.logo = logo
        self.id = str(uuid.uuid4())[:8]
        # type/logo could be an enum
        self.street_element = None
        self.element_type = None
        self.node_list.append(self)

    def __repr__(self) -> str:
        return f"{self.logo}: {self.id}, COORD: {self.coordinates}"
    
    def place_on_grid(self, dataframe):
        row,col = self.coordinates
        dataframe[row][col] = self.logo
        return dataframe
    
    def euclidean_distance(self, node):
        return dist.euclidean(self.coordinates, node.coordinates)
    
    # @abstractmethod
    def check_access(self):
        pass


class Building(Node):

    building_list = []

    def __init__(self, coordinates: tuple, logo: str, parking_space: bool) -> None:
        super().__init__(coordinates, logo)
        self.parking_space = parking_space
        self.street_element = False
        Building.building_list.append(self)

    def check_access(self):
        pass
    

class Parking(Node):

    parking_list = []
    
    def __init__(self, coordinates: tuple, logo: str) -> None:
            super().__init__(coordinates, logo)

    def check_access(self):
            pass

class Pavement(Node):

    pavement_list = []

    def __init__(self, coordinates: tuple, logo: str) -> None:
            super().__init__(coordinates, logo='Pv')
            self.street_element = True
            Pavement.pavement_list.append(self)

    def check_access(self):
        pass


In [4]:
def initialize_grid():
    # create object type numpy array with 10 rows and 10 columns
    grid = pd.DataFrame(np.chararray((10,10), itemsize=2))
    grid[:] = '*'
    for node in Node.node_list:
        grid = node.place_on_grid(grid)
    return grid


In [5]:
groceries = Building((1,4),'G', parking_space=False)
house = Building((5,4),'H', parking_space=True)
# pavement1 = Pavement((1,5),'Pv')
# pavement2 = Pavement((2,5),'Pv')
# create pavement objects
for i in range(1,7):
    pavement = Pavement((i,5),'Pv')

# create road objects
# for i in range(1,7):
#     road = Pavement((i,6),'Rd')


In [6]:

grid = initialize_grid()
print(grid)


   0   1   2   3   4   5   6  7  8  9
0  *   *   *   *   *   *   *  *  *  *
1  *   *   *   *   *   *   *  *  *  *
2  *   *   *   *   *   *   *  *  *  *
3  *   *   *   *   *   *   *  *  *  *
4  *   G   *   *   *   H   *  *  *  *
5  *  Pv  Pv  Pv  Pv  Pv  Pv  *  *  *
6  *   *   *   *   *   *   *  *  *  *
7  *   *   *   *   *   *   *  *  *  *
8  *   *   *   *   *   *   *  *  *  *
9  *   *   *   *   *   *   *  *  *  *


In [7]:
groceries.node_list

[G: 8e3a98eb, COORD: (1, 4),
 H: 6d4c7d7b, COORD: (5, 4),
 Pv: d100c58c, COORD: (1, 5),
 Pv: 80666654, COORD: (2, 5),
 Pv: d9b1d5be, COORD: (3, 5),
 Pv: 226e8278, COORD: (4, 5),
 Pv: 51fdc5e2, COORD: (5, 5),
 Pv: 242061d1, COORD: (6, 5)]

In [8]:
# Create pavement graph and add nodes
G = nx.Graph()


In [9]:

for pavement in Pavement.pavement_list:
    G.add_node(pavement)

# add pavement edges
G.add_edge(Pavement.pavement_list[0], Pavement.pavement_list[1], distance=Pavement.pavement_list[0].euclidean_distance(Pavement.pavement_list[1]))
G.add_edge(Pavement.pavement_list[1], Pavement.pavement_list[2], distance=Pavement.pavement_list[1].euclidean_distance(Pavement.pavement_list[2]))
G.add_edge(Pavement.pavement_list[2], Pavement.pavement_list[3], distance=Pavement.pavement_list[2].euclidean_distance(Pavement.pavement_list[3]))
G.add_edge(Pavement.pavement_list[3], Pavement.pavement_list[4], distance=Pavement.pavement_list[3].euclidean_distance(Pavement.pavement_list[4]))
G.add_edge(Pavement.pavement_list[4], Pavement.pavement_list[5], distance=Pavement.pavement_list[4].euclidean_distance(Pavement.pavement_list[5]))

# add edges from pavement to buildings
G.add_edge(Pavement.pavement_list[0], Building.building_list[0], distance=0.1)
G.add_edge(Pavement.pavement_list[5], Building.building_list[1], distance=0.1)


G.neighbors(Pavement.pavement_list[0])

In [10]:
for neighbour in G.neighbors(Pavement.pavement_list[0]):
    print(neighbour)
    

Pv: 80666654, COORD: (2, 5)
G: 8e3a98eb, COORD: (1, 4)


In [11]:
# show list of edges in graph
type(G.edges())
G.edges().data()


EdgeDataView([(Pv: d100c58c, COORD: (1, 5), Pv: 80666654, COORD: (2, 5), {'distance': 1.0}), (Pv: d100c58c, COORD: (1, 5), G: 8e3a98eb, COORD: (1, 4), {'distance': 0.1}), (Pv: 80666654, COORD: (2, 5), Pv: d9b1d5be, COORD: (3, 5), {'distance': 1.0}), (Pv: d9b1d5be, COORD: (3, 5), Pv: 226e8278, COORD: (4, 5), {'distance': 1.0}), (Pv: 226e8278, COORD: (4, 5), Pv: 51fdc5e2, COORD: (5, 5), {'distance': 1.0}), (Pv: 51fdc5e2, COORD: (5, 5), Pv: 242061d1, COORD: (6, 5), {'distance': 1.0}), (Pv: 242061d1, COORD: (6, 5), H: 6d4c7d7b, COORD: (5, 4), {'distance': 0.1})])

In [12]:
nx.shortest_path(G, Building.building_list[0], Building.building_list[1], weight='distance')


[G: 8e3a98eb, COORD: (1, 4),
 Pv: d100c58c, COORD: (1, 5),
 Pv: 80666654, COORD: (2, 5),
 Pv: d9b1d5be, COORD: (3, 5),
 Pv: 226e8278, COORD: (4, 5),
 Pv: 51fdc5e2, COORD: (5, 5),
 Pv: 242061d1, COORD: (6, 5),
 H: 6d4c7d7b, COORD: (5, 4)]

In [13]:
nx.shortest_path_length(G, Building.building_list[0], Building.building_list[1], weight='distance')

5.199999999999999

### Implement NodeRegister (do I need this)?

In [14]:
class NodeRegister:
    
    def __init__(self):
        self.buildings = {node.id: node for node in Building.building_list} 
        self.pavements = {node.id: node for node in Pavement.pavement_list}

In [15]:
register = NodeRegister()
register.buildings

{'8e3a98eb': G: 8e3a98eb, COORD: (1, 4),
 '6d4c7d7b': H: 6d4c7d7b, COORD: (5, 4)}